In [1]:
import os
import numpy as np
import pywt
import random
import seaborn as sns
import tensorflow as tf
from keras.layers import Add
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import (
  Input, Conv1D, BatchNormalization, Activation, GlobalAveragePooling1D, 
  Dense, Dropout, GRU, Concatenate, LayerNormalization, MultiHeadAttention, 
  Reshape, Multiply, Softmax
)
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot as plt
from utils import (
  encode_labels, check_gpu_availability, plot_loss_accuracytupian, 
  evaluate_model, plot_confusion_matrixtupian, plot_tsne, 
  plot_precision_recall_curve_multiclasstupian, plot_roc_curve_multiclasstupian, 
  AdjustLearningRateCallback, denoise2,count_labels,denoise2_iterative2,AdjustLearningRateCallback
)
from utils import plot_precision_recall_curve_multiclass,plot_roc_curve_multiclass2,calculate_g_mean,plot_confusion_matrix,plot_confusion_matrix2,plot_loss_accuracy
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv1DTranspose


c:\ProgramData\anaconda3\envs\tensorflow\lib\site-packages\scipy\__init__.py:132: UserWarning: A NumPy version >=1.21.6 and <1.28.0 is required for this version of SciPy (detected version 1.20.3)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
check_gpu_availability()

GPU 可用


In [3]:
datafilename1 = "C:\\Users\\Administrator\\Desktop\\database\\cinc2017denoise.npz"
data1 = np.load(datafilename1, allow_pickle=True)
X_train, y_train, X_val, y_val, X_test, y_test = data1['ecgstrain'], data1['labelstrain'], data1['ecgsval'], data1['labelsval'], data1['ecgstest'], data1['labelstest']

In [4]:
y_train = encode_labels(y_train)
y_test = encode_labels(y_test)
y_val= encode_labels(y_val)
y_train = to_categorical(y_train, num_classes=4)
y_val=to_categorical(y_val, num_classes=4)
y_test = to_categorical(y_test, num_classes=4)

In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, Activation, Add, GlobalAveragePooling1D, Dropout, Dense,MaxPool1D
from tensorflow.keras.models import Model

def residual_block(x, filters, kernel_size, strides):
    shortcut = x
    # 主卷积路径
    x = Conv1D(filters, kernel_size, strides=strides, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(filters, kernel_size, padding='same')(x)
    x = BatchNormalization()(x)

    # 如果shortcut的形状与x不匹配，则调整shortcut
    if shortcut.shape[-1] != x.shape[-1] or shortcut.shape[-2] != x.shape[-2]:
        shortcut = Conv1D(filters, 1, strides=strides, padding='same')(shortcut)
        shortcut = BatchNormalization()(shortcut)

    # 添加跳过连接
    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x

def resnet(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    x = Conv1D(32, 19, strides=1, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = residual_block(x, 32, 3, 2)
    x = residual_block(x, 64, 3, 2)
    x = residual_block(x, 128, 3, 2)
    x1 = GRU(128, return_sequences=False)(x)
    x2 = GRU(128, return_sequences=False)(x)
    x=x1+x2
    x = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=x)
    return model

# Example usage:
model =resnet((4500, 1), 4)
model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 4500, 1)]    0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 4500, 32)     640         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 4500, 32)     128         conv1d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 4500, 32)     0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [6]:
callback = AdjustLearningRateCallback(factor=0.1, patience=2, min_lr=1e-8)
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])
history=model.fit(X_train, y_train, batch_size=256, epochs=80, validation_data=(X_val, y_val), callbacks=[callback,early_stopping])

Epoch 1/80
55/55 [==============================] - 30s 325ms/step - loss: 0.4185 - accuracy: 0.5810 - val_loss: 0.4458 - val_accuracy: 0.5608
Epoch 2/80
55/55 [==============================] - 17s 310ms/step - loss: 0.4021 - accuracy: 0.5941 - val_loss: 0.4187 - val_accuracy: 0.5766
Epoch 3/80
55/55 [==============================] - 17s 304ms/step - loss: 0.3924 - accuracy: 0.6066 - val_loss: 0.4127 - val_accuracy: 0.5918
Epoch 4/80
55/55 [==============================] - 17s 315ms/step - loss: 0.3671 - accuracy: 0.6454 - val_loss: 0.4302 - val_accuracy: 0.5951
Epoch 5/80
55/55 [==============================] - 17s 315ms/step - loss: 0.3392 - accuracy: 0.6789 - val_loss: 0.3580 - val_accuracy: 0.6605
Epoch 6/80
55/55 [==============================] - 18s 318ms/step - loss: 0.3077 - accuracy: 0.7137 - val_loss: 0.3199 - val_accuracy: 0.7123
Epoch 7/80
55/55 [==============================] - 17s 308ms/step - loss: 0.2799 - accuracy: 0.7535 - val_loss: 0.2798 - val_accuracy: 0.7504

In [7]:
import numpy as np

def add_low_frequency_noise_multidim(data, snr, frequency_range=(0, 5), sample_rate=300):
    data_power = np.mean(data ** 2)
    noise_power = data_power / (10 ** (snr / 10))
    t = np.arange(data.shape[-1]) / sample_rate 
    noise_frequencies = np.random.uniform(frequency_range[0], frequency_range[1], size=data.shape[-1])
    noise = np.sqrt(noise_power) * np.sin(2 * np.pi * noise_frequencies * t)
    noisy_data = data + noise[None, ...] 
    
    return noisy_data

In [8]:
X_test_noisy = add_low_frequency_noise_multidim(X_test,0)

In [9]:
evaluate_model(model,X_test_noisy ,y_test)

Precision: 0.7412791302160083
Recall: 0.6238871801044448
F1 Score: 0.6671908737573503
Accuracy: 0.7727272727272727
Class 1 - Precision: 0.7337662337662337, Recall: 0.4977973568281938, F1 Score: 0.5931758530183726
Class 2 - Precision: 0.8561961563949636, Recall: 0.8607594936708861, F1 Score: 0.8584717607973423
Class 3 - Precision: 0.625154130702836, Recall: 0.6869918699186992, F1 Score: 0.6546158812136863
Class 4 - Precision: 0.75, Recall: 0.45, F1 Score: 0.5625000000000001
Class 1 Accuracy: 0.9376508447304908
Class 2 Accuracy: 0.82864038616251
Class 3 Accuracy: 0.7847948511665326
Class 4 Accuracy: 0.9943684633950121


In [10]:
X_test_noisy = add_low_frequency_noise_multidim(X_test,5)

In [11]:
evaluate_model(model,X_test_noisy ,y_test)

Precision: 0.8344491618676196
Recall: 0.7346103208711314
F1 Score: 0.7692694180843632
Accuracy: 0.8399034593724859
Class 1 - Precision: 0.7805907172995781, Recall: 0.8149779735682819, F1 Score: 0.7974137931034483
Class 2 - Precision: 0.8802318094011591, Recall: 0.9107261825449701, F1 Score: 0.8952193844138835
Class 3 - Precision: 0.7678832116788321, Recall: 0.7127371273712737, F1 Score: 0.7392832044975404
Class 4 - Precision: 0.9090909090909091, Recall: 0.5, F1 Score: 0.6451612903225806
Class 1 Accuracy: 0.9621882542236525
Class 2 Accuracy: 0.8712791633145616
Class 3 Accuracy: 0.8507642799678198
Class 4 Accuracy: 0.995575221238938


In [12]:
import numpy as np

def add_high_frequency_noise_multidim(data, snr, frequency_range=(0, 200), sample_rate=300):
    data_power = np.mean(data ** 2)
    noise_power = data_power / (10 ** (snr / 10))
    t = np.arange(data.shape[-1]) / sample_rate 
    noise_frequencies = np.random.uniform(frequency_range[0], frequency_range[1], size=data.shape[-1])
    noise = np.sqrt(noise_power) * np.sin(2 * np.pi * noise_frequencies * t)
    noisy_data = data + noise[None, ...] 
    
    return noisy_data

In [13]:
X_test_noisy = add_high_frequency_noise_multidim(X_test,0)

In [14]:
evaluate_model(model,X_test_noisy,y_test)

Precision: 0.7458225321406543
Recall: 0.6529552852010689
F1 Score: 0.6905121147410918
Accuracy: 0.7735317779565567
Class 1 - Precision: 0.6309523809523809, Recall: 0.4669603524229075, F1 Score: 0.5367088607594936
Class 2 - Precision: 0.8588312541037426, Recall: 0.871419053964024, F1 Score: 0.865079365079365
Class 3 - Precision: 0.6363636363636364, Recall: 0.6734417344173442, F1 Score: 0.654377880184332
Class 4 - Precision: 0.8571428571428571, Recall: 0.6, F1 Score: 0.7058823529411764
Class 1 Accuracy: 0.9263877715205149
Class 2 Accuracy: 0.8358809332260659
Class 3 Accuracy: 0.7888173773129525
Class 4 Accuracy: 0.99597747385358


In [15]:
X_test_noisy = add_high_frequency_noise_multidim(X_test,5)

In [16]:
evaluate_model(model,X_test_noisy ,y_test)

Precision: 0.839100683648159
Recall: 0.7229181059313609
F1 Score: 0.7657616878603206
Accuracy: 0.8386967015285599
Class 1 - Precision: 0.8181818181818182, Recall: 0.7533039647577092, F1 Score: 0.7844036697247706
Class 2 - Precision: 0.8853626943005182, Recall: 0.9107261825449701, F1 Score: 0.8978653530377668
Class 3 - Precision: 0.7437673130193906, Recall: 0.7276422764227642, F1 Score: 0.7356164383561644
Class 4 - Precision: 0.9090909090909091, Recall: 0.5, F1 Score: 0.6451612903225806
Class 1 Accuracy: 0.9621882542236525
Class 2 Accuracy: 0.8748994368463395
Class 3 Accuracy: 0.8447304907481898
Class 4 Accuracy: 0.995575221238938
